## load dataset

In [2]:
import pandas as pd

In [1]:
import xml.etree.ElementTree as ET

In [3]:
def load_xml(file):
    docs = pd.DataFrame(columns=['doc'])
    with open(file,encoding='utf-8') as f:
        f.read()
    xml = '<ROOT>' + xml + '</ROOT>'
    for doc in ET.fromstring(xml):
        docID = doc.find('DOCNO').text
        content = doc.find('HEADLINE').text + doc.find('TEXT').text
        docs_df.loc[docID] = content
    return docs
    

In [4]:
def load_stop_words(file):
    with open(file,encoding='utf-8-sig') as f:
        stop_words = [w.strip('\n') for w in f.readlines()]
    return stop_words

## Pre-processes text

In [5]:
import re

In [7]:
from nltk.stem import PorterStemmer

In [8]:
def preprocess(text):
    preprocessed_tokens = []
    stemmer = PorterStemmer()
    pattern = r"\w+"
    tokens = re.findall(pattern,text)
    for token in tokens:
        if token.lower() not in stop_words:
            preprocessed_tokens.append(stemmer.stem(token.lower()))
    return preprocessed_tokens